In [1]:
import tensorflow.compat.v1 as tf
import warnings
tf.disable_v2_behavior()
tf.set_random_seed(777)  # for reproducibility
warnings.filterwarnings('ignore')
print(tf.__version__)

Instructions for updating:
non-resource variables are not supported in the long term
2.4.0


In [2]:
import numpy as np
import random
import tensorflow as tf2 #Tensorflow 2.x

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
# normalizing data
x_train, x_test = x_train / 255.0, x_test / 255.0

# change data shape
print(x_train.shape)  # (60000, 28, 28)

(60000, 28, 28)


In [4]:
learning_rate = 0.001
batch_size = 100
training_epochs = 15
nb_classes = 10
num_epochs = 50
num_iterations = 600

In [5]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1] * x_train.shape[2])
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1] * x_test.shape[2])

y_train = tf.keras.utils.to_categorical(y_train, nb_classes)
y_test = tf.keras.utils.to_categorical(y_test, nb_classes)

#print(x_train.shape)
#print(y_train.shape)
#print(x_test.shape)
#print(y_test.shape)

In [6]:
# input place holders
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

# weights & bias for nn layers
#W1 = tf.Variable(tf.random_normal([784, 256]))

# Glorot normal initializer, also called Xavier normal initializer.
init = tf2.initializers.GlorotUniform()
W1 = tf.get_variable("W1", shape=[784, 512],
                     initializer=init)

b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.get_variable("W2", shape=[512, 512],
                     initializer=init)
b2 = tf.Variable(tf.random_normal([512]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.get_variable("W3", shape=[512, 512],
                     initializer=init)
b3 = tf.Variable(tf.random_normal([512]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

W4 = tf.get_variable("W4", shape=[512, 512],
                     initializer=init)
b4 = tf.Variable(tf.random_normal([512]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)

W5 = tf.get_variable("W5", shape=[512, 10],
                     initializer=init)
b5 = tf.Variable(tf.random_normal([10]))

In [7]:
hypothesis = tf.matmul(L4, W5) + b5


# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=tf.stop_gradient(Y)))
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [8]:
correct_prediction = tf.equal(tf.argmax(hypothesis, axis=1), tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [9]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [11]:
for epoch in range(num_epochs):
    avg_cost = 0
    for iteration in range(num_iterations):
        batch_xs, batch_ys = next_batch(batch_size, x_train, y_train)
        _, cost_val = sess.run([train, cost], feed_dict={X: batch_xs, Y: batch_ys})
        avg_cost += cost_val / num_iterations
    print(f"Epoch: {(epoch + 1):04d}, Cost: {avg_cost:.9f}")
print("Learning Finished!")
# Test model and check accuracy
print(
    "Accuracy:",
    sess.run(accuracy, feed_dict={X: x_test, Y: y_test}),
)

Epoch: 0001, Cost: 0.284764082
Epoch: 0002, Cost: 0.087248228
Epoch: 0003, Cost: 0.064932195
Epoch: 0004, Cost: 0.047785776
Epoch: 0005, Cost: 0.041803757
Epoch: 0006, Cost: 0.032195915
Epoch: 0007, Cost: 0.029448980
Epoch: 0008, Cost: 0.029348074
Epoch: 0009, Cost: 0.022692126
Epoch: 0010, Cost: 0.019520529
Epoch: 0011, Cost: 0.019212905
Epoch: 0012, Cost: 0.017805591
Epoch: 0013, Cost: 0.016777320
Epoch: 0014, Cost: 0.015431366
Epoch: 0015, Cost: 0.014211042
Epoch: 0016, Cost: 0.016392946
Epoch: 0017, Cost: 0.013450192
Epoch: 0018, Cost: 0.011738363
Epoch: 0019, Cost: 0.010579287
Epoch: 0020, Cost: 0.009116595
Epoch: 0021, Cost: 0.011191570
Epoch: 0022, Cost: 0.011976833
Epoch: 0023, Cost: 0.011886781
Epoch: 0024, Cost: 0.010685762
Epoch: 0025, Cost: 0.008775003
Epoch: 0026, Cost: 0.007722719
Epoch: 0027, Cost: 0.010104528
Epoch: 0028, Cost: 0.007042181
Epoch: 0029, Cost: 0.008600161
Epoch: 0030, Cost: 0.009707492
Epoch: 0031, Cost: 0.009356706
Epoch: 0032, Cost: 0.006893587
Epoch: 0

In [12]:
# Predict
correct_num = 0

for x in range(0, 10):
    r = random.randint(0, x_test.shape[0]-1)
    print("#",x)
    real=sess.run(tf.argmax(y_test[r]))
    predict=sess.run(tf.argmax(hypothesis, axis=1), feed_dict={X: x_test[r : r + 1]})
    print("Label: ",real)
    print("Prediction: ",predict)

    if real == predict:
        correct_num+=1
    print("Accuracy: ", correct_num/(x+1))

print("Total Accuracy: ", correct_num/(x+1))


# 0
Label:  3
Prediction:  [3]
Accuracy:  1.0
# 1
Label:  4
Prediction:  [4]
Accuracy:  1.0
# 2
Label:  3
Prediction:  [3]
Accuracy:  1.0
# 3
Label:  7
Prediction:  [4]
Accuracy:  0.75
# 4
Label:  8
Prediction:  [8]
Accuracy:  0.8
# 5
Label:  2
Prediction:  [2]
Accuracy:  0.8333333333333334
# 6
Label:  1
Prediction:  [1]
Accuracy:  0.8571428571428571
# 7
Label:  6
Prediction:  [6]
Accuracy:  0.875
# 8
Label:  0
Prediction:  [0]
Accuracy:  0.8888888888888888
# 9
Label:  8
Prediction:  [8]
Accuracy:  0.9
Total Accuracy:  0.9
